<a href="https://colab.research.google.com/github/hemanthsunny/knowledge_graphs_0.1/blob/master/first_kg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [0]:
from spacy.matcher import Matcher
from spacy.tokens import Span

In [0]:
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

In [0]:
pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [0]:
#sentences = pd.read_csv('https://github.com/hemanthsunny/knowledge_graphs_0.1/wiki_sentences_v2.csv')

# Reference: https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

In [8]:
from google.colab import files
uploaded = files.upload()

Saving wiki_sentences_v2.csv to wiki_sentences_v2.csv


In [9]:
uploaded

{'wiki_sentences_v2.csv': b'sentence\n"confused and frustrated, connie decides to leave on her own."\n"later, a woman\xe2\x80\x99s scream is heard in the distance."\nchristian is then paralyzed by an elder.\nthe temple is set on fire.\n"outside, the cult wails with him."\nit\'s a parable of a woman\'s religious awakening\xe2\x80\x94\n"c. mackenzie, and craig vincent joined the cast."\n"later, craig di francia and action bronson were revealed to have joined the cast."\nsebastian maniscalco and paul ben-victor were later revealed as being part of the cast.\nwe just tried to make the film.\nwe went through all these tests and things  \nm global was also circling to bid for the film\'s international sales rights.\ncanadian musician robbie robertson supervised the soundtrack.\nit features both original and existing music tracks.\nit is the worst reviewed film in the franchise.\nbut she injures quicksilver and accidentally kills mystique before flying away.\nmilitary forces tasked with her a

In [10]:
import io

sentences = pd.read_csv(io.BytesIO(uploaded['wiki_sentences_v2.csv']))
sentences

,sentence
0,"confused and frustrated, connie decides to leave on her own."
1,"later, a woman’s scream is heard in the distance."
2,christian is then paralyzed by an elder.
3,the temple is set on fire.
4,"outside, the cult wails with him."
...,...
4313,"confidencial also responded negatively, calling the film a barren drama, unsubtle and self-indulgent."
4314,and le parisien gave the film their highest five-star rating.
4315,"the museum collection includes 37,000 film titles, 60,000 posters, 700,000 photographs and 20,000 books."
4316,"its predecessor was the dutch historical film archive, founded in 1946."


In [11]:
sentences.shape

(4318, 1)